In [ ]:
## For colab - you must install the GPU version of this each time
!pip install faiss-gpu

In [3]:
import faiss
import os
import json
import numpy as np
import torch
import pandas as pd
from PIL import Image
import torchvision.transforms as transforms
from torchvision import datasets, models
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings('ignore')
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
#from google.cloud import storage
#from torch.utils.tensorboard import SummaryWriter
# %load_ext tensorboard
import datetime
import time
import torch.nn as nn

from torchvision.models.resnet import *
from torchvision.models.resnet import BasicBlock, Bottleneck

from google.colab import drive
drive.mount('/content/drive/')
os.chdir('drive/MyDrive/Colab Notebooks/AICORE/FAISS')

In [4]:
## Leverage the process_img function

def process_img(image):
    """
    Process an input image for use in the modified pretrained resnet50 classifier.

    Parameters
    ----------
    image : str
        The file path of the input image to be processed.

    Returns
    -------
    torch.Tensor
        A processed image represented as a PyTorch tensor with specific dimensions.
    """    
    pil_to_tensor = transforms.ToTensor()
    resize = transforms.Resize((225,225))
    img = Image.open(image).convert('RGB')

    features = pil_to_tensor(img)
    features = resize(features)
    features = torch.unsqueeze(features, dim=0)
    #print(features.shape)
    return features

In [5]:
## Read the embeddings JSON file and convert to a dictionary

with open('Faiss_API/final_embeddings_sep.json', "r") as json_file:
    data_dict = json.load(json_file)

index = faiss.IndexFlatL2(1000)   # build the index, d=size of vectors
# here we assume xb contains a n-by-d numpy matrix of type float32

## Create a flattened array of float32 vectors
embeddings_array = np.array(list(data_dict.values()), dtype='float32')
## Create a maching array of the vector ids (based on the filenames)
embeddings_ids = np.array(list(data_dict.keys()))
## Create the FAISS index by using the add function
index.add(embeddings_array)
print(index.is_trained)

True


In [6]:
## Created a classifier based on the RESNET50 pretrained model

class ItemFeatureExtractor(torch.nn.Module):
    """
    A custom nn.Module class housing the classifier which is
    based on a gpu-derived pretrained resnet50 from NVIDA torchhub
    
    Attributes
    ----------
    None
    
    
    Methods
    -------
    __init__():
        Initialises the classifier and loads the model from the torchhub
        unless it is able to detect a cached instance.
        Replaces the final layer with a 13 class output
    
    forward():
        Initiates the forward pass
        
    """        
    def __init__(self):
        super().__init__()
        self.resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
        #self.resnet50 = model
        self.resnet50.fc = torch.nn.Linear(2048,1000)

    def forward(self, X):
        return F.softmax(self.resnet50(X))

In [7]:
## Instantiate the model class as the feature extractor with the original parameters
model = ItemFeatureExtractor()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

## Load the final training weights
checkpoint = torch.load('final_weights/weights.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

Using cache found in /home/ubuntu/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


# Remove the final classification layer and make a  embedding output available
model = nn.Sequential(*list(model.resnet50.children())[:-1])

In [8]:
## Test FAISS by creating a set of query embeddings using the model output and process_image function
query_embeddings = model(process_img('pytorch_images_tv/train/appliances/16.jpg'))

In [9]:
## Flatten the output
query_embeddings = query_embeddings.view(query_embeddings.size(0), -1)

In [10]:
## Convert the output to an float 32 array
query_vector = np.array(list(query_embeddings.tolist()), dtype='float32')

In [11]:
query_vector.shape

(1, 1000)

In [12]:

## Test the index search
D, I = index.search(query_vector.reshape(1, -1), 4)

In [13]:
I

array([[1, 2, 3, 0]])

In [14]:
embeddings_ids[759]

'pytorch_images_tv/train/sportsleisure/lpi'